# Подготовка данных
необходимо создать excel таблицу с некоторыми тестовыми данными

In [142]:
import pandas as pd
import math

df = pd.read_excel("tests6.xlsx")
df.head()

,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,Ready,05/23/2024 14:48:49,Служебный пользователь для Автотестов,смоук хф
1,NaN,NaN,NaN,NaN,NaN,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
import time

start_timer = time.time()

output = df
# заполняем столбик значениями
for index, row in df.iterrows():
    if not math.isnan(df['Id'][index]):
        pass
    else:
        output['Id'][index] = df['Id'][index - 1]
        
    
    
end_timer = time.time()
print(end_timer - start_timer)
output.to_excel("output.xlsx", index = False)
output.head()

0.0031690597534179688


,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,Ready,05/23/2024 14:48:49,Служебный пользователь для Автотестов,смоук хф
1,544031.0,NaN,NaN,NaN,NaN,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,544031.0,NaN,NaN,NaN,NaN,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,544031.0,NaN,NaN,NaN,NaN,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,544031.0,NaN,NaN,NaN,NaN,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# spaCy

```conda install spacy```

Тест маленькой модели

In [124]:
import spacy
from spacy.lang.ru.examples import sentences 

# запуск таймера
start_timer = time.time()
# подгрузка языковой модели
nlp = spacy.load("ru_core_news_sm")

# функция векторизации
def vectorize_text(text, model):
    doc = model(text)
    return doc.vector

print(str(df))

vector_table = vectorize_text(str(df), nlp)

print(vector_table)
# vector_id = vectorize_text(str(df['Id']), nlp)
# vector_direction = vectorize_text(str(df['Direction']), nlp)
# vector_steps = vectorize_text(str(df['Steps']), nlp)
# 
# print(str(df))
# 
# print(str(df['Id']), "\n")
# print(str(df['Direction']), "\n")
# print(str(df['Steps']), "\n")
# print(type(vector_id))
# 
# print(vector_id, "\n")
# print(vector_direction, "\n")
# print(vector_steps, "\n")

end_timer = time.time()
print(end_timer - start_timer)

          Id   Direction                 Section  \
0   544031.0  AI_Отладка  Пул задач пользователя   
1        NaN         NaN                     NaN   
2        NaN         NaN                     NaN   
3        NaN         NaN                     NaN   
4        NaN         NaN                     NaN   
..       ...         ...                     ...   
59       NaN         NaN                     NaN   
60       NaN         NaN                     NaN   
61       NaN         NaN                     NaN   
62       NaN         NaN                     NaN   
63       NaN         NaN                     NaN   

                    TestCaseName Automated  \
0   Контролер. Исполнить задачу      False   
1                            NaN       NaN   
2                            NaN       NaN   
3                            NaN       NaN   
4                            NaN       NaN   
..                           ...       ...   
59                           NaN       NaN   
60     

In [125]:
#print(len(vector_id), len(vector_direction), len(vector_steps))
print(len(vector_table))

96
